## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [82]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [0]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [84]:
import os
print(os.getcwd())
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./service-account.json"

/content


In [0]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums

def score_review(review):
  client = language_v1.LanguageServiceClient()
  document = {
      "content": review,
      "type": enums.Document.Type.PLAIN_TEXT,
      "language": "en"
  }
  response = client.analyze_sentiment(
      document, encoding_type=enums.EncodingType.UTF8
  )

  sentiment_score = response.document_sentiment.score

  return 'pos' if sentiment_score >= 0.0 else 'neg'

### Score each review

Now, we can use the function you defined to score each of the reviews

In [86]:
print(f"Scoring positive reviews {len(reviews_pos)}")
results_pos = []
for i, review in enumerate(reviews_pos):
    if i % 100 == 0:
      print(i)
    result = score_review(review)
    results_pos.append(result)

Scoring positive reviews 1000
0
100
200
300
400
500
600
700
800
900


In [87]:
print(f"Scoring negative reviews {len(reviews_neg)}")
results_neg = []
for i, review in enumerate(reviews_neg):
    if i % 100 == 0:
      print(i)
    result = score_review(review)
    results_neg.append(result)

Scoring negative reviews 1000
0
100
200
300
400
500
600
700
800
900


### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [88]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 93.30000000000001% correct
Negative reviews: 65.8% correct
Overall accuracy: 79.55% correct
